In [1]:
from ultralytics import YOLO,RTDETR
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# %% [markdown]
# ## 3. Download and Load YOLO Model

# %%
# Choose your model: yolov8n.pt (nano), yolov8s.pt (small), yolov8m.pt (medium), etc.

model_name = 'models/rtdetr.pt'

print(f"Loading YOLO model: {model_name}")
print("This will download the model if not already cached...\n")

# Load model (downloads automatically if not present)
model = RTDETR(model_name)

print(f"✓ Model loaded successfully!")
print(f"Model type: {model_name}")
print(f"Model task: {model.task}")


# %%
import time
from IPython.display import display, clear_output

def run_camera_detection(model, camera_index, display_size, confidence_threshold):
    """
    Run real-time object detection on camera feed
    
    Args:
        model: YOLO model to use
        camera_index: Camera device index (usually 0 for built-in webcam)
        display_size: Display resolution (width, height)
        confidence_threshold: Minimum confidence for detections
    
    Controls:
        - Press 'q' in the camera window to stop
        - Or interrupt the kernel
    """
    
    # Open camera
    cap = cv2.VideoCapture(camera_index)
    
    if not cap.isOpened():
        print("❌ Error: Could not open camera")
        print("Try changing camera_index (0, 1, 2, etc.)")
        return
    
    # Set camera resolution
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, display_size[0])
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, display_size[1])
    
    print("🎥 Camera opened successfully!")
    print("📹 Press 'q' in the camera window to stop")
    print("=" * 50)
    
    frame_count = 0
    start_time = time.time()
    
    try:
        while True:
            # Read frame from camera
            ret, frame = cap.read()
            
            if not ret:
                print("❌ Error: Could not read frame")
                break
            
            # Run YOLO inference
            results = model(frame, conf=confidence_threshold, verbose=False)
            result = results[0]
            
            # Get annotated frame
            annotated_frame = result.plot()
            
            # Calculate FPS
            frame_count += 1
            elapsed_time = time.time() - start_time
            fps = frame_count / elapsed_time if elapsed_time > 0 else 0
            
            # Add FPS and detection count to frame
            cv2.putText(annotated_frame, f'FPS: {fps:.1f}', (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(annotated_frame, f'Detections: {len(result.boxes)}', (10, 70),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            # Display frame
            cv2.imshow('YOLO Real-Time Detection', annotated_frame)
            
            # Print detection info every 30 frames
            if frame_count % 30 == 0:
                print(f"Frame {frame_count} | FPS: {fps:.1f} | Detections: {len(result.boxes)}")
                if len(result.boxes) > 0:
                    detected = {}
                    for box in result.boxes:
                        cls_id = int(box.cls[0])
                        class_name = model.names[cls_id]
                        detected[class_name] = detected.get(class_name, 0) + 1
                    print(f"  Objects: {detected}")
            
            # Break loop with 'q' key
            if cv2.waitKey(1) & 0xFF == ord('q'):
                print("\n⏹️  Stopping camera detection...")
                break
                
    except KeyboardInterrupt:
        print("\n⏹️  Detection interrupted by user")
    
    finally:
        # Cleanup
        cap.release()
        cv2.destroyAllWindows()
        
        # Final statistics
        total_time = time.time() - start_time
        avg_fps = frame_count / total_time if total_time > 0 else 0
        print("=" * 50)
        print(f"✅ Detection completed!")
        print(f"📊 Total frames: {frame_count}")
        print(f"⏱️  Total time: {total_time:.1f}s")
        print(f"🎯 Average FPS: {avg_fps:.1f}")

# Run camera detection
# Adjust parameters as needed:
# - camera_index: 0 for default camera, try 1, 2 for external cameras
# - confidence_threshold: 0.5 (default), lower for more detections, higher for fewer
camera_index = 1
treshold = 0.6
run_camera_detection(model, camera_index,confidence_threshold=treshold, display_size=(1920, 1080))

# %% [markdown]
# ## 9. Batch Processing (Optional)

# %%
def process_multiple_images(image_paths):
    """Process multiple images and display results in a grid"""
    n_images = len(image_paths)
    cols = 2
    rows = (n_images + 1) // 2
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
    axes = axes.flatten() if n_images > 1 else [axes]
    
    for idx, img_path in enumerate(image_paths):
        results = model(img_path)
        annotated_img = results[0].plot()
        annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
        
        axes[idx].imshow(annotated_img_rgb)
        axes[idx].axis('off')
        axes[idx].set_title(f'Image {idx+1}: {len(results[0].boxes)} detections')
    
    # Hide extra subplots
    for idx in range(n_images, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

# Example usage:
# image_list = ['image1.jpg', 'image2.jpg', 'image3.jpg']
# process_multiple_images(image_list)

# %% [markdown]
# ## 10. Model Information

# %%
# Display model capabilities
print("Available classes:", len(model.names))
print("\nClass names:")
for idx, name in model.names.items():
    print(f"  {idx}: {name}")



KeyboardInterrupt: 